In [ ]:
import aiohttp
import asyncio
import nest_asyncio
import pandas as pd

from mainnet_launch.constants import *
from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import *
import numpy as np

nest_asyncio.apply()

_rate_limit = asyncio.Semaphore(10)
_session: aiohttp.ClientSession | None = None


async def get_session() -> aiohttp.ClientSession:
    global _session
    if _session is None or _session.closed:
        _session = aiohttp.ClientSession()
    return _session


async def fetch_swap_quote(
    chain_id: int,
    sell_token: str,
    buy_token: str,
    sell_amount: int,
    system_name: str = "gen3",
    slippage_bps: int = 50,
    include_sources: str = "",
    exclude_sources: str = "Bebop",
    sell_all: bool = True,
    timeout_ms: int = None,
    transfer_to_caller: bool = True,
) -> dict:
    async with _rate_limit:
        url = "https://swaps-pricing.tokemaklabs.com/swap-quote-v2"
        payload = {
            "chainId": chain_id,
            "systemName": system_name,
            "slippageBps": slippage_bps,
            "taker": DEAD_ADDRESS,
            "sellToken": sell_token,
            "buyToken": buy_token,
            "sellAmount": sell_amount,
            "includeSources": include_sources,
            "excludeSources": exclude_sources,
            "sellAll": sell_all,
            "timeoutMS": str(timeout_ms) if timeout_ms is not None else "",
            "transferToCaller": str(transfer_to_caller),
        }

        session = await get_session()
        async with session.post(url, json=payload) as resp:
            try:
                resp.raise_for_status()
                data = await resp.json()
                data.update(payload)
            except Exception as e:
                data = {"error": str(e), **payload}
                raise e
        return data


async def fetch_quote_size_df(
    token_orms: list[Tokens], sizes: list[float], autopool: AutopoolConstants
) -> pd.DataFrame:

    tasks = []

    for size in sizes:
        for t in token_orms:

            tasks.append(
                fetch_swap_quote(
                    chain_id=t.chain_id,
                    sell_token=t.token_address,
                    buy_token=autopool.base_asset,
                    sell_amount=int(size * 10**t.decimals),
                )
            )
    quotes = await asyncio.gather(*tasks)
    quote_df = pd.DataFrame.from_records(quotes)
    return quote_df


async def fetch_quote_df(
    token_orms: list[Tokens], base_sell_amount: float, autopool: AutopoolConstants
) -> pd.DataFrame:
    # launch all requests in parallel
    quotes = await asyncio.gather(
        *(
            fetch_swap_quote(
                chain_id=t.chain_id,
                sell_token=t.token_address,
                buy_token=autopool.base_asset,
                sell_amount=int(base_sell_amount * 10**t.decimals),
            )
            for t in token_orms
        )
    )
    return pd.DataFrame.from_records(quotes)


from mainnet_launch.pages.autopool_exposure.allocation_over_time import _fetch_tvl_by_asset_and_destination


autopool = AUTO_DOLA

safe_value_by_destination, safe_value_by_asset, backing_value_by_destination, quantity_by_asset = (
    _fetch_tvl_by_asset_and_destination(autopool)
)
token_orms = get_full_table_as_orm(Tokens, where_clause=Tokens.chain_id == autopool.chain.chain_id)

# note this introduces some latency, but not a big deal imo
quantity_by_asset


latest_quantity_by_assets = quantity_by_asset.iloc[-1]
tokens_to_get_quotes_for = [t for t in token_orms if t.symbol in latest_quantity_by_assets.index ]
tokens_to_get_quotes_for



token_symbol_to_token_orm = {t.symbol: t for t in token_orms}
all_quotes = []
for token_symbol, normalized_quantity in latest_quantity_by_assets.items():
    this_token_orm: Tokens = token_symbol_to_token_orm[token_symbol]
    unscaled_quantity = int(normalized_quantity * (10 ** (this_token_orm.decimals) ))
    print(token_symbol_to_token_orm[token_symbol],  normalized_quantity, unscaled_quantity)


    for scale in range(1,10):
        percent_of_assets_to_liquidate =  scale / 10

        quote = fetch_swap_quote(
                    chain_id=autopool.chain.chain_id,
                    sell_token=this_token_orm.token_address,
                    buy_token=autopool.base_asset,
                    sell_amount=int(unscaled_quantity * percent_of_assets_to_liquidate)
                )
        all_quotes.append(quote)
        break

a_quote = await all_quotes[0]
a_quote

2025-06-25 10:54:34.832 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-25 10:54:34.878 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 10:54:34.939 
  command:

    streamlit run /Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-06-25 10:54:34.939 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 10:54:34.940 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 10:54:34.940 No runtime found, using MemoryCacheStorageManager
2025-06-25 10:54:35.446 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 10:54:35.446 Thread 'Thread-4

[Tokens(token_address='0x0655977FEb2f289A4aB78af67BAB0d17aAb84367', chain_id=1, symbol='scrvUSD', name='Savings crvUSD', decimals=18),
 Tokens(token_address='0x57aB1E0003F623289CD798B1824Be09a793e4Bec', chain_id=1, symbol='reUSD', name='Resupply USD', decimals=18),
 Tokens(token_address='0x865377367054516e17014CcdED1e7d814EDC9ce4', chain_id=1, symbol='DOLA', name='Dola USD Stablecoin', decimals=18),
 Tokens(token_address='0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD', chain_id=1, symbol='sUSDS', name='Savings USDS', decimals=18),
 Tokens(token_address='0xBC6DA0FE9aD5f3b0d58160288917AA56653660E9', chain_id=1, symbol='alUSD', name='Alchemix USD', decimals=18),
 Tokens(token_address='0xb45ad160634c528Cc3D2926d9807104FA3157305', chain_id=1, symbol='sDOLA', name='Staked Dola', decimals=18),
 Tokens(token_address='0x66a1E37c9b0eAddca17d3662D6c05F4DECf3e110', chain_id=1, symbol='USR', name='Resolv USD', decimals=18),
 Tokens(token_address='0x9D39A5DE30e57443BfF2A8307A4256c8797A3497', chain_id=1

In [ ]:

# quote_df =  pd.DataFrame.from_records(await asyncio.gather( *all_quotes))

# quotes_df

Tokens(token_address='0x865377367054516e17014CcdED1e7d814EDC9ce4', chain_id=1, symbol='DOLA', name='Dola USD Stablecoin', decimals=18) 142110.23921932117 142110239219321178947584
Tokens(token_address='0x66a1E37c9b0eAddca17d3662D6c05F4DECf3e110', chain_id=1, symbol='USR', name='Resolv USD', decimals=18) 0.0 0
Tokens(token_address='0xBC6DA0FE9aD5f3b0d58160288917AA56653660E9', chain_id=1, symbol='alUSD', name='Alchemix USD', decimals=18) 92269.45978062181 92269459780621807124480
Tokens(token_address='0x15700B564Ca08D9439C58cA5053166E8317aa138', chain_id=1, symbol='deUSD', name='deUSD', decimals=18) 0.0 0
Tokens(token_address='0x57aB1E0003F623289CD798B1824Be09a793e4Bec', chain_id=1, symbol='reUSD', name='Resupply USD', decimals=18) 1108113.2814035171 1108113281403517050814464
Tokens(token_address='0xb45ad160634c528Cc3D2926d9807104FA3157305', chain_id=1, symbol='sDOLA', name='Staked Dola', decimals=18) 2770902.9539302196 2770902953930219845582848
Tokens(token_address='0xa3931d71877C0E7a3148

ClientResponseError: 400, message='Bad Request', url='https://swaps-pricing.tokemaklabs.com/swap-quote-v2'

In [ ]:
quote_df

In [ ]:
latest_quantity_by_assets

In [ ]:
latest_quantity_by_assets

In [ ]:
break

In [ ]:
import quotes_at_scale
import ast

import numpy as np
import plotly.express as px
import pandas as pd
import plotly.io as pio

pio.templates.default = None

df = pd.read_csv(
    "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/mainnet_launch/data_fetching/quotes/sonicUSD_scale_quotes.csv"
)

df["min_buy_scaled"] = df["minBuyAmount"].apply(lambda x: int(x) / 1e6 if x > 0 else np.nan)
df["buy_scaled"] = df["buyAmount"].apply(lambda x: int(x) / 1e6 if x > 0 else np.nan)

# scale sellAmount by its own per-row decimals
df["sell_amount_scaled"] = df.apply(lambda row: int(row["sellAmount"]) / (10 ** row["decimals"]), axis=1)

df["buy_ratio"] = df["buy_scaled"] / df["sell_amount_scaled"]
df["min_buy_ratio"] = df["min_buy_scaled"] / df["sell_amount_scaled"]

price_df = df.pivot(columns="symbol", index="sell_amount_scaled", values="buy_scaled")
ratio_df = df.pivot(columns="symbol", index="sell_amount_scaled", values="buy_ratio")
ratio_df

In [ ]:
# not symetrical down,

# all the near the same time?

# not really certian there

In [ ]:
px.scatter(ratio_df, title="sonic_USD_ratio", log_x=True)

In [ ]:
df[df["symbol"] == "escUSD1"]

In [ ]:
price_df

In [ ]:
# quotes fail, for > 100 frxUSD

In [ ]:
df.columns

In [ ]:
df = pd.read_csv("quotes_records2.csv")
df["fullQuoteDetails_parsed"] = df["fullQuoteDetails"].apply(ast.literal_eval)


def extract_block(row):

    if "data" in row["fullQuoteDetails_parsed"]:
        return int(row["fullQuoteDetails_parsed"]["data"]["blockNumber"])
    elif "quote" in row["fullQuoteDetails_parsed"]:
        return int(row["fullQuoteDetails_parsed"]["quote"]["blockNumber"])


df["block"] = df.apply(extract_block, axis=1)
df

In [ ]:
df["symbol"].value_counts()

In [ ]:
# did something change this minute with the sell able toekns?

In [ ]:
df["min_buy_scaled"] = df["minBuyAmount"].apply(lambda x: int(x) / 1e6)
df["buy_scaled"] = df["buyAmount"].apply(lambda x: int(x) / 1e6)
df["sell_amount_scaled"] = df.apply(lambda row: int(row["sellAmount"]) / (10 ** row["decimals"]), axis=1)

In [ ]:
df[["symbol", "min_buy_scaled", "buy_scaled", "sell_amount_scaled"]]

In [ ]:
import plotly.express as px

px.scatter(df, x=df["block"], y="min_buy_scaled", color="symbol")

In [ ]:
px.scatter(df, x=df.index, y="buy_scaled", color="symbol")